# Sentiment Analysis on IMDB Review Data USing LSTM

In [1]:
!pip install kaggle

Importing the dependencies

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential  #Neural network
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


Data Collection KAggle api

In [4]:
kaggle_dict=json.load(open('kaggle.json'))

In [5]:
# setup kaggle creds as env variables
os.environ["KAGGLE_USERNAME"]=kaggle_dict['username']
os.environ["KAGGLE_KEY"]=kaggle_dict['key']

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [10]:
#unzip dataset
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  zip_ref.extractall()

In [11]:
!ls

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading Data

In [12]:
data=pd.read_csv('IMDB Dataset.csv')

In [13]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
data.shape

(50000, 2)

In [17]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [18]:
data.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)

/tmp/ipython-input-18-52814477.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)


In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [21]:
#split data into training and testing data
train_data,test_data=train_test_split(data, test_size=0.2,random_state=42)

In [22]:
train_data.shape

(40000, 2)

In [23]:
test_data.shape

(10000, 2)

Data Preprocessing

In [25]:
#Tokenize text data
tokenizer = Tokenizer(num_words=5000) #Used to convert words or phrases to numbers
tokenizer.fit_on_texts(train_data['review']) #For no leakage
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200) #to make sure all the input data are of the same length
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [26]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [27]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [28]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']